In [12]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql import SQLContext, Row  
from pyspark.sql import functions as F
sc =SparkContext()
spark = SparkSession(sc)
sqlContext=SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-1-26b0206b2e89>:5 

In [14]:
precipitation = sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/precipitation-readings.csv")

parts = precipitation.map(lambda a: a.split(';'))
precReadingsRow = parts.map(lambda x: (x[0], x[1], int(x[1].split("-")[0]), int(x[1].split("-")[1]), x[2], float(x[3]), x[4]))
precReadingsString = ["station", "date", "year", "month", "time", "precip", "quality"]
schemaPrecReadings = sqlContext.createDataFrame(precReadingsRow, precReadingsString)

rdd = sc.textFile("C:/Users/quartermaine/Documents/Big Data Analytics Labs/stations-Ostergotland.csv")

parts = rdd.map(lambda a: a.split(';'))
statOstRow = parts.map(lambda x: (x[0], x[1]))
statOstString = ["station", "name"]
schemaStatOst = sqlContext.createDataFrame(statOstRow, statOstString)


schemaPrecReadings = schemaPrecReadings.filter( (schemaPrecReadings['year'] >= 1993) & (schemaPrecReadings['year'] <= 2016) )

schemaPrecReadings = schemaPrecReadings.join(schemaStatOst, 'station', 'inner')

schemaPrecReadings.groupBy('station', 'year', 'month').agg(F.sum('precip')).groupBy('year', 'month').agg(F.avg('sum(precip)').alias('avg_monthly_precipitation')).orderBy(['year', 'month'], ascending=[0, 0]).show()

+----+-----+-------------------------+
|year|month|avg_monthly_precipitation|
+----+-----+-------------------------+
|2016|    7|                      0.0|
|2016|    6|                  47.6625|
|2016|    5|       29.250000000000007|
|2016|    4|       26.900000000000006|
|2016|    3|       19.962500000000006|
|2016|    2|                  21.5625|
|2016|    1|       22.325000000000003|
|2015|   12|                   28.925|
|2015|   11|        63.88750000000002|
|2015|   10|                   2.2625|
|2015|    9|                    101.3|
|2015|    8|       26.987499999999997|
|2015|    7|       119.09999999999994|
|2015|    6|        78.66250000000002|
|2015|    5|                   93.225|
|2015|    4|       15.337499999999999|
|2015|    3|        42.61250000000001|
|2015|    2|                   24.825|
|2015|    1|        59.11250000000003|
|2014|   12|        35.46250000000001|
+----+-----+-------------------------+
only showing top 20 rows

